In [92]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

BORDER = "==============================================================="

In [93]:
def load_data(filename):
    raw_text = open(filename).read()
    text_list = raw_text.strip().split('\n')
    return text_list

In [94]:
def remove_int(text):
    '''
    Remove numbers from strings
    '''
    new_text = []
    for line in text:
        # Remove integer values
        no_digits = ''.join([i for i in line if not i.isdigit()])

        # Remove punctuation
        new_text.append(no_digits)
    return new_text

In [95]:
def remove_empty(text):
    '''
    Removes all empty string
    '''
    new_text = []
    for line in text:
        if line != '':
            new_text.append(line)
    return new_text

In [96]:
def process_data_RNN(text):
    '''
    Create fixed length training sequences of length 40 char from the sonnet
    corpus.

    Input: Text file in the form of list of words

    Output: X, Y, dataX, dataY, int_to_char, n_vocab
    '''
    
    new_text_list = remove_int(text)
    new_text_list = remove_empty(new_text_list)
    new_text = '\n'.join(new_text_list)
    
    print(BORDER)
    print("Processed text: ", new_text)
    print(BORDER)

    # create mapping of unique chars to integers, and a reverse mapping
    chars = sorted(list(set(new_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    # summarize the loaded data
    n_chars = len(new_text)
    n_vocab = len(chars)
    print("Total Characters: ", n_chars)
    print("Total Vocab: ", n_vocab)
    print("Int to char: ", int_to_char)

    # prepare the dataset of input to output pairs encoded as integers
    seq_length = 40
    dataX = []
    dataY = []
    for i in range(0, n_chars - seq_length):
        seq_in = new_text[i:i + seq_length]
        seq_out = new_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)
    print("Total Patterns: ", n_patterns)
    print(BORDER)

    X = np.zeros((n_patterns, seq_length, n_vocab))
    y = np.zeros((n_patterns, n_vocab))
    for i, sentence in enumerate(dataX):
        for t, ind in enumerate(sentence):
            X[i, t, ind] = 1
        y[i, dataY[i]] = 1


    return X, y, dataX, dataY, int_to_char, char_to_int

In [97]:
def train_LSTM(X, y, verbose=0):
    '''
    Takes training data X and Y and returns the fitted LSTM model

    Input:
        X : a list of sequences of int
        Y : one-hot encoding of the int coming after the sequence
    '''

    # Take a submit of sequences
#     X = X[0::10]
#     y = y[0::10]

    # define the LSTM model
    model = Sequential()
    model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

    # fit the model
    model.fit(X, y, epochs=60, batch_size=128, verbose=verbose)
    return model

In [98]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [99]:
def generate_text(model, dataX, int_to_char, verbose=0):
    '''
    Given model, dataX, int_to_char, n_vocab returns generated_text using
    predict function

    Input:
        model: the LSTM model that we trained
        dataX: list of sequences
        int_to_char: a dictionary matching interger to specific character
        n_vocab: number of unique characters we have

    Output: generate_text as string

    '''
    print(BORDER)
    print("Generating text")
    n_vocab = len(int_to_char)
    diversity = 0.5

    # pick a random seed
    start = np.random.randint(0, len(dataX) - 1)
    pattern = dataX[start]
    seq = [int_to_char[value] for value in pattern]
    size = len(pattern)

    if verbose == 1:
        print("Seed: ", ''.join(seq))

    # generate characters
    for i in range(600):
        # Create and normalize x to be input of RNN
        x = np.zeros((1, size, n_vocab))
        for t, char in enumerate(pattern):
            x[0, t, char] = 1

        # Make prediction using trained model
        prediction = model.predict(x, verbose=verbose)[0]
        index = sample(prediction, diversity)

        # Convert prediction to character
        result = int_to_char[index]

        # Add prediction to pattern and set to size 40
        pattern.append(index)
        pattern = pattern[1:1 + size]

        # Add result to seq
        seq.append(result)

        if verbose == 1:
            print("pred: ", prediction[0])
            print("selected index: ", index)
            print("selected char: ", result)
            print("new pattern: ",
                  ''.join([int_to_char[value] for value in pattern]))

    # Return seq as string
    return ''.join(seq)

In [100]:
def save_textfile(filename, text):
    '''
    Given filename and text, save text in file

    Input: filename and text as string
    '''
    f = open(filename, 'w')
    f.write(text)
    f.close()
    return 0

In [101]:
file = 'data/shakespeare.txt'
save = 'generated/shakespeare_2.txt'
verbose = 1

text_list = load_data(file)
(X, y, dataX, dataY, int_to_char, char_to_int) = process_data_RNN(text_list)

Processed text:  From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
  Pity the world, or else this glutton be,
  To eat the world's due, by the grave and thee.
                   
When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a tattered weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say within thine own deep sunken eyes,
Were an all-eating shame, and thriftless prai

Total Patterns:  97309


In [102]:

model = train_LSTM(X, y, verbose=verbose)

Epoch 1/60
9731/9731 [==============================] - 23s 2ms/step - loss: 3.1156 - acc: 0.1937
Epoch 2/60
9731/9731 [==============================] - 19s 2ms/step - loss: 3.0115 - acc: 0.1967
Epoch 3/60
9731/9731 [==============================] - 17s 2ms/step - loss: 2.9670 - acc: 0.1983
Epoch 4/60
9731/9731 [==============================] - 16s 2ms/step - loss: 2.8810 - acc: 0.2068
Epoch 5/60
9731/9731 [==============================] - 16s 2ms/step - loss: 2.7561 - acc: 0.2467
Epoch 6/60
9731/9731 [==============================] - 17s 2ms/step - loss: 2.6296 - acc: 0.2769
Epoch 7/60
9731/9731 [==============================] - 16s 2ms/step - loss: 2.5134 - acc: 0.3073
Epoch 8/60
9731/9731 [==============================] - 17s 2ms/step - loss: 2.4282 - acc: 0.3229
Epoch 9/60
9731/9731 [==============================] - 16s 2ms/step - loss: 2.3598 - acc: 0.3406
Epoch 10/60
9731/9731 [==============================] - 16s 2ms/step - loss: 2.3078 - acc: 0.3474
Epoch 11/60
9731/97

In [103]:
generated = generate_text(model, dataX, int_to_char, verbose=verbose)
save_textfile(save, generated)
print(generated)

Generating text
Seed:  hine,
And I my self am mortgaged to thy 
1/1 [==============================] - 0s 442ms/step
pred:  2.2667201e-07
selected index:  53
selected char:  s
new pattern:  ine,
And I my self am mortgaged to thy s
1/1 [==============================] - 0s 6ms/step
pred:  3.394084e-07
selected index:  49
selected char:  o
new pattern:  ne,
And I my self am mortgaged to thy so
1/1 [==============================] - 0s 6ms/step
pred:  6.6663524e-05
selected index:  52
selected char:  r
new pattern:  e,
And I my self am mortgaged to thy sor
1/1 [==============================] - 0s 8ms/step
pred:  9.96245e-05
selected index:  39
selected char:  e
new pattern:  ,
And I my self am mortgaged to thy sore
1/1 [==============================] - 0s 10ms/step
pred:  0.00012809876
selected index:  6
selected char:  ,
new pattern:  
And I my self am mortgaged to thy sore,
1/1 [==============================] - 0s 10ms/step
pred:  0.9941379
selected index:  0
selected char:  

new pa

pred:  0.09585329
selected index:  39
selected char:  e
new pattern:   whet watt in howe, sure ert wast counte
1/1 [==============================] - 0s 8ms/step
pred:  0.027482398
selected index:  52
selected char:  r
new pattern:  whet watt in howe, sure ert wast counter
1/1 [==============================] - 0s 7ms/step
pred:  0.019261923
selected index:  1
selected char:   
new pattern:  het watt in howe, sure ert wast counter 
1/1 [==============================] - 0s 5ms/step
pred:  0.002356286
selected index:  49
selected char:  o
new pattern:  et watt in howe, sure ert wast counter o
1/1 [==============================] - 0s 8ms/step
pred:  1.06405205e-05
selected index:  57
selected char:  w
new pattern:  t watt in howe, sure ert wast counter ow
1/1 [==============================] - 0s 6ms/step
pred:  1.8550934e-05
selected index:  6
selected char:  ,
new pattern:   watt in howe, sure ert wast counter ow,
1/1 [==============================] - 0s 7ms/step
pred:  0.9340181
sel

pred:  0.9377433
selected index:  0
selected char:  

new pattern:  ingie hath moke to my aid thee arthere,

1/1 [==============================] - 0s 7ms/step
pred:  0.00015649207
selected index:  12
selected char:  A
new pattern:  ngie hath moke to my aid thee arthere,
A
1/1 [==============================] - 0s 5ms/step
pred:  3.202751e-07
selected index:  48
selected char:  n
new pattern:  gie hath moke to my aid thee arthere,
An
1/1 [==============================] - 0s 7ms/step
pred:  3.8643606e-07
selected index:  38
selected char:  d
new pattern:  ie hath moke to my aid thee arthere,
And
1/1 [==============================] - 0s 8ms/step
pred:  2.272063e-06
selected index:  1
selected char:   
new pattern:  e hath moke to my aid thee arthere,
And 
1/1 [==============================] - 0s 8ms/step
pred:  1.7873908e-05
selected index:  55
selected char:  u
new pattern:   hath moke to my aid thee arthere,
And u
1/1 [==============================] - 0s 6ms/step
pred:  9.284577e-0

pred:  2.124145e-05
selected index:  35
selected char:  a
new pattern:  te thut both sofrows of or mowh,
And sea
1/1 [==============================] - 0s 7ms/step
pred:  2.595443e-05
selected index:  43
selected char:  i
new pattern:  e thut both sofrows of or mowh,
And seai
1/1 [==============================] - 0s 7ms/step
pred:  9.435346e-06
selected index:  53
selected char:  s
new pattern:   thut both sofrows of or mowh,
And seais
1/1 [==============================] - 0s 7ms/step
pred:  6.1489627e-06
selected index:  1
selected char:   
new pattern:  thut both sofrows of or mowh,
And seais 
1/1 [==============================] - 0s 9ms/step
pred:  2.3563648e-07
selected index:  54
selected char:  t
new pattern:  hut both sofrows of or mowh,
And seais t
1/1 [==============================] - 0s 7ms/step
pred:  1.0661846e-10
selected index:  42
selected char:  h
new pattern:  ut both sofrows of or mowh,
And seais th
1/1 [==============================] - 0s 6ms/step
pred:  5.40977

pred:  3.7464977e-07
selected index:  49
selected char:  o
new pattern:  s ay my sove aid and loke,
Thes broot ho
1/1 [==============================] - 0s 8ms/step
pred:  4.238337e-06
selected index:  47
selected char:  m
new pattern:   ay my sove aid and loke,
Thes broot hom
1/1 [==============================] - 0s 7ms/step
pred:  3.2722915e-06
selected index:  1
selected char:   
new pattern:  ay my sove aid and loke,
Thes broot hom 
1/1 [==============================] - 0s 7ms/step
pred:  2.2813465e-05
selected index:  38
selected char:  d
new pattern:  y my sove aid and loke,
Thes broot hom d
1/1 [==============================] - 0s 7ms/step
pred:  5.048644e-06
selected index:  49
selected char:  o
new pattern:   my sove aid and loke,
Thes broot hom do
1/1 [==============================] - 0s 6ms/step
pred:  1.3901849e-06
selected index:  54
selected char:  t
new pattern:  my sove aid and loke,
Thes broot hom dot
1/1 [==============================] - 0s 4ms/step
pred:  2.4814

...

pred:  3.5060404e-07
selected index:  53
selected char:  s
new pattern:  tore thee ofream woll mome of tote,
Whas
1/1 [==============================] - 0s 6ms/step
pred:  3.2275263e-06
selected index:  1
selected char:   
new pattern:  ore thee ofream woll mome of tote,
Whas 
1/1 [==============================] - 0s 7ms/step
pred:  8.427135e-07
selected index:  59
selected char:  y
new pattern:  re thee ofream woll mome of tote,
Whas y
1/1 [==============================] - 0s 6ms/step
pred:  2.9262269e-08
selected index:  49
selected char:  o
new pattern:  e thee ofream woll mome of tote,
Whas yo
1/1 [==============================] - 0s 8ms/step
pred:  1.3658075e-09
selected index:  55
selected char:  u
new pattern:   thee ofream woll mome of tote,
Whas you
1/1 [==============================] - 0s 8ms/step
pred:  5.0967607e-08
selected index:  52
selected char:  r
new pattern:  thee ofream woll mome of tote,
Whas your
1/1 [==============================] - 0s 7ms/step
pred:  1.116

pred:  1.3008528e-06
selected index:  35
selected char:  a
new pattern:  breath menth monet wile sell:
o me ho ba
1/1 [==============================] - 0s 7ms/step
pred:  0.00033902994
selected index:  43
selected char:  i
new pattern:  reath menth monet wile sell:
o me ho bai
1/1 [==============================] - 0s 7ms/step
pred:  0.0011343947
selected index:  52
selected char:  r
new pattern:  eath menth monet wile sell:
o me ho bair
1/1 [==============================] - 0s 7ms/step
pred:  0.0013027374
selected index:  39
selected char:  e
new pattern:  ath menth monet wile sell:
o me ho baire
1/1 [==============================] - 0s 7ms/step
pred:  4.7158548e-05
selected index:  38
selected char:  d
new pattern:  th menth monet wile sell:
o me ho baired
1/1 [==============================] - 0s 6ms/step
pred:  0.0005623834
selected index:  1
selected char:   
new pattern:  h menth monet wile sell:
o me ho baired 
1/1 [==============================] - 0s 8ms/step
pred:  0.00014

pred:  0.000117878044
selected index:  1
selected char:   
new pattern:   thot beacend greed:
But pross ind hork 
1/1 [==============================] - 0s 7ms/step
pred:  1.5463933e-06
selected index:  49
selected char:  o
new pattern:  thot beacend greed:
But pross ind hork o
1/1 [==============================] - 0s 8ms/step
pred:  6.2151184e-06
selected index:  48
selected char:  n
new pattern:  hot beacend greed:
But pross ind hork on
1/1 [==============================] - 0s 5ms/step
pred:  2.9049543e-06
selected index:  1
selected char:   
new pattern:  ot beacend greed:
But pross ind hork on 
1/1 [==============================] - 0s 6ms/step
pred:  1.46212715e-05
selected index:  38
selected char:  d
new pattern:  t beacend greed:
But pross ind hork on d
1/1 [==============================] - 0s 6ms/step
pred:  2.6130732e-05
selected index:  39
selected char:  e
new pattern:   beacend greed:
But pross ind hork on de
1/1 [==============================] - 0s 6ms/step
pred:  7.3

1/1 [==============================] - 0s 7ms/step
pred:  2.8506678e-05
selected index:  54
selected char:  t
new pattern:   deare.
                 
Tha s yer th t
1/1 [==============================] - 0s 7ms/step
pred:  6.36516e-07
selected index:  42
selected char:  h
new pattern:  deare.
                 
Tha s yer th th
1/1 [==============================] - 0s 8ms/step
pred:  9.8799596e-08
selected index:  39
selected char:  e
new pattern:  eare.
                 
Tha s yer th the
1/1 [==============================] - 0s 9ms/step
pred:  8.2633405e-06
selected index:  1
selected char:   
new pattern:  are.
                 
Tha s yer th the 
1/1 [==============================] - 0s 9ms/step
pred:  1.5898899e-05
selected index:  36
selected char:  b
new pattern:  re.
                 
Tha s yer th the b
1/1 [==============================] - 0s 8ms/step
pred:  3.2938826e-07
selected index:  39
selected char:  e
new pattern:  e.
                 
Tha s yer th the be
1/1 [========

hine,
And I my self am mortgaged to thy sore,
Than whet watt in howe, sure ert wast counter ow,
And ingie hath moke to my aid thee arthere,
And ury erte thut both sofrows of or mowh,
And seais the thas ay my sove aid and loke,
Thes broot hom doth the the art tho goul,
Thac of the byoun the warte the woth the wort,
To thand the bee thend of broud thate,
And sost the cencenou pour sichine deess,
Be tore thee ofream woll mome of tote,
Whas your you breath menth monet wile sell:
o me ho baired theer thot beacend greed:
But pross ind hork on dears on deare.
                 
Tha s yer th the bess be seart in moved gaect,
And siss in sees
